<a href="https://colab.research.google.com/github/futartup/S8-assignment/blob/master/copy_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tqdm import tqdm
import torch
import torch.nn.functional as F

In [0]:



def test_model(model, device, 
               test_loader, criterion, 
               e, sample_count,
               correct_predicted, false_predicted, 
               test_losses, test_acc):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()*data.size(0)  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            result = pred.eq(target.view_as(pred))
            if e == 2:
              for i,val in enumerate(result):
                status = val.item()
                if status:
                  if len(correct_predicted) < sample_count:
                    correct_predicted.append({
                        'prediction': pred[i],
                        'label': list(target.view_as(pred))[i],
                        'image': data[i]
                    })
                else:
                  if len(false_predicted) < sample_count:
                    false_predicted.append({
                        'prediction': pred[i],
                        'label': list(target.view_as(pred))[i],
                        'image': data[i]
                    })
            correct += result.sum().item()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    #pbar.set_description(desc= f'Loss={test_loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss